<a href="https://colab.research.google.com/github/86HenriqueSilva/mega_sena/blob/main/01_Mega_Sena_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Números de Sorteio com Rede Neural

Este notebook utiliza uma rede neural para prever números de sorteio com base em dados históricos. Abaixo está uma explicação rápida de cada seção do código:

1. **Importação de Bibliotecas**: Carrega as bibliotecas necessárias para manipulação de dados, treinamento do modelo e visualização.

2. **Função de Pós-processamento**: Corrige previsões para garantir que não haja números duplicados.

3. **Carregamento dos Dados**: Baixa os dados de sorteios, remove linhas inválidas e a coluna de data.

4. **Pré-processamento dos Dados**: Divide os dados em conjuntos de treinamento e teste e normaliza os dados de entrada.

5. **Construção e Treinamento do Modelo**: Cria uma rede neural e a treina usando o conjunto de dados.

6. **Avaliação e Pós-processamento**: Avalia o modelo no conjunto de teste, faz ajustes nas previsões e exibe os resultados.

Siga as etapas abaixo para executar o código e testar o modelo.


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm
import gdown

# Função para pós-processamento das previsões
def postprocess_predictions(predictions):
    # Calcular o número de ocorrências de cada valor
    counts = np.bincount(predictions, minlength=61)  # Definindo o tamanho mínimo do array
    # Iterar sobre as previsões para cada bola
    for i in range(len(predictions)):
        # Verificar se há repetições
        while counts[predictions[i]] > 1:
            # Encontrar um novo valor não utilizado ainda
            new_value = np.random.randint(1, 61)
            while counts[new_value] > 0:
                new_value = np.random.randint(1, 61)
            # Substituir a previsão repetida pelo novo valor
            predictions[i] = new_value
            # Atualizar o contador
            counts[predictions[i]] += 1
    return predictions

# URL do arquivo
data_url = 'https://drive.google.com/uc?id=1t28TCezmfyBc5X_3-JwqN-FxkV63M-t5'

# Baixar o arquivo e carregar como DataFrame
data = pd.read_csv(data_url)

# Remover linhas com valores não numéricos
data = data.dropna()

# Remover a coluna de data
data = data.drop(columns=['Data'])

# Pré-processamento dos dados
numerical_columns = data.select_dtypes(include=[np.number]).columns
X = data.drop(columns=['Bola 1','Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6'])
y = data[['Bola 1','Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=512)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Construir e treinar o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6)  # 6 saídas para prever os 6 números
])
model.compile(optimizer='adam', loss='mean_squared_error')

epochs = 5000
batch_size = 128
with tqdm(total=epochs, desc="Treinamento da Rede Neural") as pbar:
    for epoch in range(epochs):
        pbar.update(1)
        model.fit(X_train_scaled, y_train, batch_size=batch_size, verbose=0)
        pbar.set_postfix({'Epoch': epoch + 1})

# Avaliar o modelo
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)

# Arredondar as previsões para valores inteiros dentro do intervalo de 1 a 60
y_pred_int = np.clip(np.round(y_pred), 1, 60).astype(int)

# Postprocessamento das previsões
y_pred_int_postprocessed = postprocess_predictions(y_pred_int[0])

# Exibir as previsões pós-processadas para o próximo concurso
print("Previsões pós-processadas para o próximo sorteio:")
for i, pred in enumerate(y_pred_int_postprocessed):
    print(f"Bola {i+1}: {pred}")

# Calcular e exibir o erro médio absoluto no conjunto de teste com duas casas decimais
print("Erro médio absoluto no conjunto de teste: {:.2f}".format(mae))

Treinamento da Rede Neural:   0%|          | 0/5000 [00:00<?, ?it/s]

18/18 [==============================] - 0s 3ms/step
Previsões pós-processadas para o próximo sorteio:
Bola 1: 25
Bola 2: 26
Bola 3: 15
Bola 4: 2
Bola 5: 30
Bola 6: 33
Erro médio absoluto no conjunto de teste: 14.75
